In [209]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
import time
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display
from riotwatcher import RiotWatcher
import requests

api_file = os.path.join('..','..','apikeys','LOL_api_key.txt')
with open(api_file, 'r') as fin:
    watcher = RiotWatcher(fin.readline())

# Get matchids for masters** tier players
because there is no api call to just get a bunch matches I have to do some weird stuff. My strategy is: get challenger player list from the region -> getsummoner by playerid to get accountid -> get recent matches from each player -> check duplicates -> get match details

One note here is that the master tier isn't a constant size across regions unlike challenger. This is a possible source of bias in the data? Probably fine since there shouldn't be too too much difference across regions.

In [211]:
regions = ['RU', 'KR', 'BR1', 'OC1', 'JP1', 'NA1', 'EUN1', 'EUW1', 'TR1', 'LA1', 'LA2']
#regions = ['LA1', 'LA2']

gameid_list = {}
league_items = {}

#get last challenger games for each player in each region (purge duplicates)
#first loop through regions
for region in regions:
    
    #get all 'items' in the challenger league for solo queue (not sure about 'flex' queues?)
    #changed to master tier becuase that is the highest until mid january
    league_items[region] = watcher.league.masters_by_queue(region, 'RANKED_SOLO_5x5')['entries']
    
    #initialize gameid_list items
    gameid_list[region] = []

#loop through all items in each region's league to get matchid's
for region in regions:
    print(region)

    #get 100 matches from each player  
    for entry in league_items[region]:

        #convert player_id(aka summoner_id) to account_id
        account_id = watcher.summoner.by_id(region, entry['playerOrTeamId'])['accountId']

        #don't keep all matches in memory or on disk - we will get full match details later
        #queue = 420 is the code for 5x5_solo_queue
        match_list = watcher.match.matchlist_by_account(region,
                                     account_id,
                                     queue=420,
                                     begin_index = 1,
                                     end_index=100)['matches']
            
        #keep gameid_list in memory for later use and for easy purging of duplicates
        for match in match_list:
            gameid_list[region].append(match['gameId']) 

        #purge duplicates from gameid_list
        gameid_list[region] = list(set(gameid_list[region]))

       

RU
KR
BR1
OC1
JP1
NA1
EUN1
EUW1
TR1
LA1
LA2


In [216]:
with open(os.path.join('LOL_data','masters_tier','saved'+time.strftime('%Y-%m-%d')+'.json'), 'w') as fp:
    json.dump(gameid_list, fp)

# Get match details

In [217]:
lol_df = pd.DataFrame()
region_list = []
gameid_load_list = []
with open(os.path.join('LOL_Data','masters_tier','saved2018-12-22.json'),'r') as fp:
    for key, item in json.load(fp).items():
        region_list.extend([key] * len(item))
        gameid_load_list.extend(item)
        
lol_df['region'] = region_list
lol_df['gameid'] = gameid_load_list
lol_df.describe()

,gameid
count,1.784010e+05
mean,2.784957e+09
std,9.401525e+08
min,1.898462e+08
25%,2.033551e+09
50%,2.922288e+09
75%,3.463426e+09
max,3.872611e+09


In [218]:
#reindex randomly so that we don't loop through regions sequentially
#this way we more efficiently use the api calls we're allotted
lol_df = lol_df.reindex(np.random.permutation(lol_df.index))

In [219]:
blue_win = []
blue_champs = []
red_champs = []
bad_element_list = []
region_list = []
gameid_list = []
for i,row in lol_df[:].iterrows():
    
    #NOTE: Summoner spell info is included in match_details but may be hard to keep linked to participants
    #NOTE 2: Timeline info. can give timeline info on many things for teams and individual participants
    #   this could be used for training  a real time win probability model.
    try:
        match_details = watcher.match.by_id(row['region'],row['gameid'])   
    except requests.exceptions.HTTPError as err:
        print(err)
        print('Error at row:',row['gameid'])
        print('element:',i)
        bad_element_list.append(i)
        continue
    
    #append region_list and gameid_list so we don't have to
    #deal with wrong bad region, gameid
    region_list.append(row['region'])
    gameid_list.append(row['gameid'])
    
    #blue win is true if teamid=100 (blue) and this same team won
    blue_win.append(int(match_details['teams'][0]['win']=='Win' and match_details['teams'][0]['teamId']==100))

    #build team champion lineups and append to columns
    blue_champ_row = []
    red_champ_row = []
    for participant in match_details['participants']:
        if participant['teamId'] == 100:
            blue_champ_row.append(participant['championId'])
        elif participant['teamId'] ==200:
            red_champ_row.append(participant['championId'])
        else:
            print('Illegal team value!',participant['teamId'])
            break
    blue_champs.append(blue_champ_row)
    red_champs.append(red_champ_row)    

503 Server Error: Service Unavailable for url: https://kr.api.riotgames.com/lol/match/v3/matches/3392594045
Error at row: 3392594045
element: 19244
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/620565660
Error at row: 620565660
element: 92945
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3642151133
Error at row: 3642151133
element: 175703
500 Server Error: Server Error for url: https://kr.api.riotgames.com/lol/match/v3/matches/3460229731
Error at row: 3460229731
element: 20064
404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2910795126
Error at row: 2910795126
element: 155005
503 Server Error: Service Unavailable for url: https://eun1.api.riotgames.com/lol/match/v3/matches/2053927246
Error at row: 2053927246
element: 111361
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3642098624
Error at row: 3642098624
element: 172285
40

404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2908058635
Error at row: 2908058635
element: 160249
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/624944562
Error at row: 624944562
element: 104798
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/620180747
Error at row: 620180747
element: 94891
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/622326926
Error at row: 622326926
element: 105652
429 Client Error: Too Many Requests for url: https://br1.api.riotgames.com/lol/match/v3/matches/1489342003
Error at row: 1489342003
element: 69314
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/626142403
Error at row: 626142403
element: 79537
404 Client Error: Not Found for url: https://eun1.api.riotgames.com/lol/match/v3/matches/3824014623
Error at row: 3824014623
element: 127098
500 Server Error: Ser

404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/619994288
Error at row: 619994288
element: 82250
404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2920064473
Error at row: 2920064473
element: 142779
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3642158630
Error at row: 3642158630
element: 176944
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3566928429
Error at row: 3566928429
element: 172210
503 Server Error: Service Unavailable for url: https://br1.api.riotgames.com/lol/match/v3/matches/1499087321
Error at row: 1499087321
element: 47803
500 Server Error: Server Error for url: https://kr.api.riotgames.com/lol/match/v3/matches/3462431234
Error at row: 3462431234
element: 11302
503 Server Error: Service Unavailable for url: https://kr.api.riotgames.com/lol/match/v3/matches/3434474936
Error at row: 3434474936
element: 41987
404 

404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/622179424
Error at row: 622179424
element: 102019
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3559595347
Error at row: 3559595347
element: 173417
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3603910203
Error at row: 3603910203
element: 175593
404 Client Error: Not Found for url: https://euw1.api.riotgames.com/lol/match/v3/matches/2910230681
Error at row: 2910230681
element: 141713
429 Client Error: Too Many Requests for url: https://eun1.api.riotgames.com/lol/match/v3/matches/2068408536
Error at row: 2068408536
element: 122246
404 Client Error: Not Found for url: https://na1.api.riotgames.com/lol/match/v3/matches/3836919856
Error at row: 3836919856
element: 87999
404 Client Error: Not Found for url: https://tr1.api.riotgames.com/lol/match/v3/matches/3579685312
Error at row: 3579685312
element: 173990
404 Client Er

In [220]:
print(len(gameid_list)+len(bad_element_list))

178401


# Process match data and save to .csv

## Drop elements's from lol_df for which we got a HTTPError 
(all of these should be stored in bad_element_list). 
Not needed since we can just remake lol_df with the saved gameid+region lists

In [187]:
#drop bad gameid's from list
for bad_element in bad_element_list:
    lol_df = lol_df.drop(bad_element)

In [188]:
print(len(lol_df[:int(len(lol_df)/50)]))
print(len(bad_element_list))
print(len(blue_win))

4844
16
4828


## Process champion lists into np arrays.
Then add blue_win, blue_champ_array, and red_champ_array to lol_df.

In [221]:
#transform list of champion lists to np.array of champions.
#Shape is match x team (number of matches x 5 for each team)
blue_champ_array = np.zeros(shape=(len(blue_champs),5))
red_champ_array = np.zeros(shape=(len(red_champs),5))

if len(red_champ_array)!=len(blue_champ_array):
    pass

for i in range(len(blue_champ_array)):
    blue_champ_array[i] = blue_champs[i]
    red_champ_array[i] = red_champs[i]

print(blue_champ_array.shape)
print(red_champ_array.shape)

(178201, 5)
(178201, 5)


In [222]:
lol_df_processed = pd.DataFrame()

#Add radiant_win column to Dataframe
lol_df_processed['blue_win'] = np.array(blue_win)

#Add gameid
#Note: gameid can be duplicates across regions
lol_df_processed['gameid'] = gameid_list

#add region
lol_df_processed['region'] = region_list

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(blue_champs)[1])):
    print(i)
    lol_df_processed['blue_champs'+str(i)] = blue_champ_array[:,i]
    lol_df_processed['red_champs'+str(i)] = red_champ_array[:,i]

0
1
2
3
4


In [223]:
#save data to csv file so we have it on disk
lol_df_processed.to_csv(os.path.join('LOL_data',
                                'masters_tier',
                                'saved'+
                                time.strftime('%Y-%m-%d')+
                                '.csv'))